In [12]:
from bs4 import BeautifulSoup
import numpy
import requests as res
from urllib.parse import urlparse, parse_qs, urljoin
import json
import openpyxl
import time
import random
import queue
import threading
random.seed(1287)


# daily webtoon

def crawler_daily_webtoon():
    get_daily_webtoon = res.get("http://comic.naver.com/webtoon/weekday.nhn")
    soup = BeautifulSoup(get_daily_webtoon.content, "lxml")
    tag = soup.select(".daily_all a.title")
    webtoon_list={}
    for a in tag:
        webtoon_list[a.get("title")]=str("http://comic.naver.com") + a.get("href")

    # webtoon_list = [{"title": a.get("title"), "link": str("http://comic.naver.com") + a.get("href")} for a in tag]

    return webtoon_list

# all_episode

def cralwer_episode(webtoon):
    target_webtoon = list()
    page_num = 1
    is_unlast = True
    while is_unlast:
        link = webtoon + "&page=" + str(page_num)
        get_page = res.get(link)
        page = BeautifulSoup(get_page.content, "lxml")
        a_tags = page.select(".viewList td.title a")
        for a_tag in a_tags:
            episode = a_tag.text.replace("\n", "").replace("\r", "").replace("\t", "")
            episode_url = "http://comic.naver.com" + str(a_tag.get("href"))

            if episode_url not in target_webtoon:
                target_webtoon.append(episode_url)

            else:
                is_unlast = False

        page_num += 1
    return target_webtoon

def get_comments(soup, url):
        title = soup.title.text.split(':')[0]
        titleId = str(parse_qs(urlparse(url).query)['titleId'][0])  # 웹툰 고유아이디값
        no = str(parse_qs(urlparse(url).query)['no'][0])  # 웹툰 그 회차 페이지

        comment_url = 'http://comic.naver.com' + '/comment/comment.nhn?titleId=' + titleId + '&no=' + no
        objectId = titleId + '_' + no
        sub_title = soup.select(".tit_area h3")
        sub_title = sub_title[0].text
        page_count = 1

        u = 'http://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery1113012327623800394427_1489937311100&lang=ko&country=KR&objectId=' + objectId + '&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&sort=NEW&_=1489937311112'

        comment_list = list()

        while True:
            comment_url = u + "&page=" + str(page_count)
            header = {
                "Host": "apis.naver.com",
                "Referer": "http://comic.naver.com/comment/comment.nhn?titleId=" + titleId + "&no=" + no,
                "Content-Type": "application/javascript"
            }
            # time.sleep(random.uniform(0.3, 0.8))
            rq = res.get(comment_url, headers=header)

            soup = BeautifulSoup(rq.content, 'lxml')

            try:
                content_text = soup.select('p')[0].text
                one = content_text.find('(') + 1
                two = content_text.find(');')
                content = json.loads(content_text[one:two])

                comments = content['result']['commentList']

                for comment in comments:
                     comment_list.append({"sub_title":sub_title,
                                          "title": title,
                                          "comment":comment["contents"],
                                          "sympathy_count":comment["sympathyCount"],
                                          "episode_num":no})
                   #  print(comment["contents"])
                if not len(comments):
                    return comment_list
                    
                else:
                    page_count += 1
            except:
                print ("not found \t %10s %d page" % (sub_title,page_count))
                page_count += 1



In [2]:
class Input_Thread(threading.Thread):
    def __init__(self, input_queue, out_queue):
        threading.Thread.__init__(self)
        self.input_queue = input_queue
        self.out_queue = out_queue

    def run(self):
        while True:
            url = self.input_queue.get()
            #             rt=res.get(url)
            #             soup=BeautifulSoup(rt.content,"lxml")
            self.out_queue.put(url)

            self.input_queue.task_done()


class Output_Thread(threading.Thread):
    def __init__(self,out_queue,last_queue):
        threading.Thread.__init__(self)
        self.out_queue = out_queue
        self.last_queue=last_queue
    def run(self):
        while True:
            data = self.out_queue.get()
            rt = res.get(data)
            soup = BeautifulSoup(rt.content, "lxml")

            # print("[" + str(self.idx) + " : " + str(data) + "]")
            # time.sleep(random.uniform(2.1, 2.8))
            last_queue.put(get_comments(soup, data))
            self.last_queue.task_done()
            self.out_queue.task_done()


In [6]:
a=crawler_daily_webtoon()

In [7]:
a

{'10월 28일': 'http://comic.naver.com/webtoon/list.nhn?titleId=702423&weekday=wed',
 '1인용 기분': 'http://comic.naver.com/webtoon/list.nhn?titleId=703835&weekday=sun',
 'Doll 체인지': 'http://comic.naver.com/webtoon/list.nhn?titleId=693444&weekday=thu',
 'Penguin loves Mev': 'http://comic.naver.com/webtoon/list.nhn?titleId=169080&weekday=fri',
 '가담항설': 'http://comic.naver.com/webtoon/list.nhn?titleId=670144&weekday=thu',
 '가우스전자 시즌3': 'http://comic.naver.com/webtoon/list.nhn?titleId=675554&weekday=fri',
 '간 떨어지는 동거': 'http://comic.naver.com/webtoon/list.nhn?titleId=699415&weekday=thu',
 '간질간질': 'http://comic.naver.com/webtoon/list.nhn?titleId=702165&weekday=tue',
 '감염자': 'http://comic.naver.com/webtoon/list.nhn?titleId=675823&weekday=sun',
 '갓 오브 하이스쿨': 'http://comic.naver.com/webtoon/list.nhn?titleId=318995&weekday=fri',
 '강변살다': 'http://comic.naver.com/webtoon/list.nhn?titleId=578108&weekday=fri',
 '갸오오와 사랑꾼들': 'http://comic.naver.com/webtoon/list.nhn?titleId=670151&weekday=sat',
 '걸어서 30분':

In [13]:
fire_hogu=cralwer_episode(a["열정호구"])

In [14]:
fire_hogu

['http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=65&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=64&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=63&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=62&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=61&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=60&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=59&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=58&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=57&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=56&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=55&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=54&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=

In [9]:

a=crawler_daily_webtoon()


fire_hogu=cralwer_episode(a["열정호구"])

input_queue=queue.Queue()
out_queue=queue.Queue()
last_queue=queue.Queue()


for i in fire_hogu[-3:]:

    th = Input_Thread(input_queue, out_queue)
    th.setDaemon(True)
    th.start()

for epi in fire_hogu:
    input_queue.put(epi)

for epi1 in range(3):
    out = Output_Thread(out_queue,last_queue)
    out.setDaemon(True)
    out.start()

input_queue.join()
out_queue.join()
last_queue.join()

TypeError: string indices must be integers

In [11]:
comment=[]
while not last_queue.empty():
    
    comment.append(last_queue.get())

___
# start of dataframe to excel

In [12]:
import pandas as pd

In [13]:
first=pd.DataFrame()
for one_episode in comment:
    data=pd.DataFrame(one_episode)
    first=pd.concat([first,data],ignore_index=True)
    
    

In [14]:
len(comment[0])

547

In [20]:
first.to_excel('out_put1.xlsx', sheet_name='sheet1')
